## ENTREGA 2: CREACIÓN DE BBDD EN MONGODB

En este notebook se muestran los pasos seguidos para la realización de la practica, desde la creación de la base de datos y la colección correspondiente, hasta la ingesta de datos de 100 jugadoras. Finalmente, también se muestran a modo de ejemplo algunas consultas analíticas sobre la base de datos (junto con los tiempos de respuesta) para comprobar que todo funciona correctamente.

En primer lugar, importamos la librería pymongo, que nos va a permitir conectarnos a una instancia de MongoDB. Para ello, únicamente hay que utilizar la clase MongoClient y especificar la IP y el puerto en el que corre MongoDB. En este caso, mi máquina está conectada a un contenedor de Docker en el que está corriendo MongoDB ya que mapée los puertos al crear el contenedor.

In [172]:
from pymongo import MongoClient

client = MongoClient("localhost", 27017)

Una vez conectado a MongoDB, creo una base de datos llamada player_database y le asigno la variable db.

In [173]:
db = client.player_database


Dentro de esta base de datos, creo una colección llamada players y le asigno una variable con el mismo nombre. Será en esta colección donde se almacenen las instancias de datos.

In [174]:
players = db.players


Con el fin de simplificar la creación de datos y su ingesta, he creado la función generate_random_data. En ella, asigno valores aleatorios a cada dato a partir de listas con posibles valores. También hay algunos datos como el número de goles para los que simplemente escojo un valor numérico aleatorio. Una vez creo todos los datos de una jugadora, se suben a la base de datos. Este proceso se repite 100 veces.

In [175]:
import random

def generate_random_data():
    nombres = ["Aisha", "Sakura", "Maria", "Fatima", "Sofia", 
               "Carmen", "Mei", "Emily", "Amara", "Ingrid", 
               "Anya", "Priya", "Zofia", "Lucia", "Chloe"]

    apellidos_1 = ["Garcia", "Muller", "Kim", "Smith", "Khan", 
                   "Dupont", "Santos", "Yamamoto", "Haddad", "Ivanov", 
                   "Andersson", "Patel", "Oliveira", "Kowalski", "O'Connor"]
    
    apellidos_2 = ["Garcia", "Muller", "Kim", "Smith", "Khan", 
                   "Dupont", "Santos", "Yamamoto", "Haddad", "Ivanov", 
                   "Andersson", "Patel", "Oliveira", "Kowalski", "O'Connor"]
    
    posiciones = ['Defensa', 'Centrocampista', 'Delantera']

    equipos = ["FC Barcelona", "Real Madrid", "Bayern Munich", "Juventus", 
               "Manchester City", "Paris Saint-Germain", "Chelsea FC", 
               "Ajax", "Borussia Dortmund", "Inter de Milan", 
               "Arsenal", "Tottenham Hotspur", "Napoli", "Atletico de Madrid", "AC Milan"]

    nacionalidades = ["Espaniola", "Alemana", "Japonesa", "Estadounidense", "Argentina", 
                      "China", "Italiana", "Nigeriana", "Francesa", "India", 
                      "Noruega", "Rusa", "Polaca", "Mexicana", "Egipcia"]
    
    pierna_dominante = ['Izquierda', 'Derecha', 'Ambas']

    # Diccionario para asignar liga correcta en base a equipo.
    equipos_y_ligas = {"FC Barcelona": "La Liga",
                       "Real Madrid": "La Liga",
                       "Bayern Munich": "Bundesliga",
                       "Juventus": "Serie A (Italia)",
                       "Manchester City": "Premier League",
                       "Paris Saint-Germain": "Ligue 1",
                       "Chelsea FC": "Premier League",
                       "Ajax": "Eredivisie",
                       "Borussia Dortmund": "Bundesliga",
                       "Inter de Milán": "Serie A",
                       "Arsenal": "Premier League",
                       "Tottenham Hotspur": "Premier League",
                       "Napoli": "Serie A",
                       "Atletico de Madrid": "La Liga",
                       "AC Milan": "Serie A"}
    
    capitana = ['Si', 'No']

    for i in range(100):

        equipo = random.choice(equipos)
        año_comienzo = random.randint(2020, 2023)
        edad = random.randint(16, 21)
        altura = random.randint(150, 190)
        peso = random.randint(50, 70)
        dorsal = random.randint(2, 30)
        temporadas_en_equipo = random.randint(1, 3)

        # Por cada año recogemos las estadísticas de la jugadora.
        stats_año = {}
        for año in range(año_comienzo, 2024):
            stats_año[str(año)] = {
                'goles': random.randint(0, 20),
                'asistencias': random.randint(0, 20),
                'porcentaje disparos gol': random.randint(50, 100),
                'porcentaje pases acertados': random.randint(50, 100),
                'porcentaje duelos aereos ganados': random.randint(25, 100),
                'velocidad maxima': random.randint(25, 30),
                'minutos jugados': random.randint(900, 4000),
                'tarjetas amarillas': random.randint(0, 10),
                'tarjetas rojas': random.randint(0, 5),
                'numero lesiones': random.randint(0, 2),
                'partidos como internacional': random.randint(0, 20)
            }

        # Asignamos los datos generados a la jugadora.
        jugadora = {
            'identificador': i+1,
            'nombre': random.choice(nombres),
            'apellido_1': random.choice(apellidos_1),
            'apellido_2': random.choice(apellidos_2),
            'posicion': random.choice(posiciones),
            'equipo': equipo,
            'liga': equipos_y_ligas.get(equipo),
            'dorsal': dorsal,
            'temporadas en equipo': temporadas_en_equipo,
            'capitana': random.choice(capitana),
            'nacionalidad': random.choice(nacionalidades),
            'anio de inicio': año_comienzo,
            'edad': edad,
            'altura': altura,
            'peso': peso,
            'pierna dominante': random.choice(pierna_dominante),
            'estadisticas': stats_año
        }

        # Subir info de cada jugadora a la base de datos.
        players.insert_one(jugadora)

Una vez creada la función, la llamamos.

In [176]:
generate_random_data()

Para comprobar que todo se ha subido correctamente, muestro información de una de las jugadoras.

In [179]:
import json

doc = players.find_one()
doc_json = json.dumps(doc, default=str, indent=4)
print(doc_json)

{
    "_id": "674769af1d7a4a76639b3497",
    "identificador": 1,
    "nombre": "Zofia",
    "apellido_1": "Oliveira",
    "apellido_2": "Garcia",
    "posicion": "Centrocampista",
    "equipo": "Borussia Dortmund",
    "liga": "Bundesliga",
    "dorsal": 24,
    "temporadas en equipo": 3,
    "capitana": "No",
    "nacionalidad": "Egipcia",
    "anio de inicio": 2021,
    "edad": 20,
    "altura": 157,
    "peso": 54,
    "pierna dominante": "Derecha",
    "estadisticas": {
        "2021": {
            "goles": 9,
            "asistencias": 13,
            "porcentaje disparos gol": 87,
            "porcentaje pases acertados": 56,
            "porcentaje duelos aereos ganados": 94,
            "velocidad maxima": 30,
            "minutos jugados": 2951,
            "tarjetas amarillas": 7,
            "tarjetas rojas": 1,
            "numero lesiones": 1,
            "partidos como internacional": 17
        },
        "2022": {
            "goles": 9,
            "asistencias": 7,
 

## CONSULTAS

Una vez se ha explicado cómo se crea la base de datos y cómo se ha realizado la ingesta de datos, en este apartado voy realizar una serie de consultas analíticas orientadas al caso de uso que se detalla en el documento adjunto en la entrega. Para cada consulta, se muestra además del resultado, el tiempo requerido para obtener la respuesta. En primer lugar quiero obtener información acerca de aquellas jugadoras que comenzaron en el fútbol profesional después del año 2021.

In [180]:
import time

# Registrar tiempo inicial.
start_time = time.time()

# Realizar la consulta para jugadoras que comenzaron en el fútbol profesional a partir del 2021.
resultados = players.find({"anio de inicio": {'$gt':2021}})

# Registrar tiempo final.
end_time = time.time()

# Convertir resultados a una lista y mostrarlos.
resultados_lista = list(resultados)

for doc in resultados_lista:
    print(doc)

# Calcular y mostrar el tiempo de ejecución.
print(f"Tiempo de ejecución: {end_time - start_time:.4f} segundos")

{'_id': ObjectId('674769af1d7a4a76639b349a'), 'identificador': 4, 'nombre': 'Amara', 'apellido_1': 'Patel', 'apellido_2': 'Kim', 'posicion': 'Centrocampista', 'equipo': 'Tottenham Hotspur', 'liga': 'Premier League', 'dorsal': 5, 'temporadas en equipo': 2, 'capitana': 'Si', 'nacionalidad': 'Italiana', 'anio de inicio': 2022, 'edad': 19, 'altura': 188, 'peso': 68, 'pierna dominante': 'Derecha', 'estadisticas': {'2022': {'goles': 1, 'asistencias': 18, 'porcentaje disparos gol': 72, 'porcentaje pases acertados': 78, 'porcentaje duelos aereos ganados': 49, 'velocidad maxima': 25, 'minutos jugados': 3080, 'tarjetas amarillas': 6, 'tarjetas rojas': 0, 'numero lesiones': 1, 'partidos como internacional': 8}, '2023': {'goles': 10, 'asistencias': 15, 'porcentaje disparos gol': 96, 'porcentaje pases acertados': 76, 'porcentaje duelos aereos ganados': 68, 'velocidad maxima': 30, 'minutos jugados': 1827, 'tarjetas amarillas': 6, 'tarjetas rojas': 0, 'numero lesiones': 2, 'partidos como internaciona

Vemos que el tiempo de ejecución es muy bajo. Esto permite ver que MongoDB sería una herramienta adecuada para un caso real de este tipo en el que se tuviera un mayor volumen de datos (pongamos varios miles de jugadoras en lugar de 100), ya que los tiempos de respuesta seguirían siendo mínimos.

Por otro lado, también se podría querer filtrar en función del equipo en el que juega la jugadora. A modo de ejemplo, vamos a mostrar únicamente aquellas que juegan en el Manchester City.

In [181]:
# Registrar tiempo inicial.
start_time = time.time()

# Realizar la consulta para jugadoras que juegan en el equipo deseado.
resultados = players.find({"equipo": 'Manchester City'})

# Registrar tiempo final.
end_time = time.time()

# Convertir resultados a una lista y mostrarlos.
resultados_lista = list(resultados)

for doc in resultados_lista:
    print(doc)

# Calcular y mostrar el tiempo de ejecución.
print(f"Tiempo de ejecución: {end_time - start_time:.4f} segundos")

{'_id': ObjectId('674769af1d7a4a76639b34aa'), 'identificador': 20, 'nombre': 'Carmen', 'apellido_1': 'Oliveira', 'apellido_2': 'Kim', 'posicion': 'Defensa', 'equipo': 'Manchester City', 'liga': 'Premier League', 'dorsal': 15, 'temporadas en equipo': 1, 'capitana': 'No', 'nacionalidad': 'Alemana', 'anio de inicio': 2020, 'edad': 18, 'altura': 165, 'peso': 59, 'pierna dominante': 'Derecha', 'estadisticas': {'2020': {'goles': 7, 'asistencias': 16, 'porcentaje disparos gol': 84, 'porcentaje pases acertados': 79, 'porcentaje duelos aereos ganados': 41, 'velocidad maxima': 30, 'minutos jugados': 1225, 'tarjetas amarillas': 8, 'tarjetas rojas': 1, 'numero lesiones': 0, 'partidos como internacional': 14}, '2021': {'goles': 0, 'asistencias': 16, 'porcentaje disparos gol': 70, 'porcentaje pases acertados': 87, 'porcentaje duelos aereos ganados': 69, 'velocidad maxima': 25, 'minutos jugados': 2981, 'tarjetas amarillas': 4, 'tarjetas rojas': 1, 'numero lesiones': 0, 'partidos como internacional': 

Pasemos ahora a filtrar por nacionalidad para mostrar sólo jugadoras japonesas.

In [182]:
# Registrar tiempo inicial.
start_time = time.time()

# Realizar la consulta para jugadoras de una nacionalidad específica.
resultados = players.find({"nacionalidad": 'Japonesa'})

# Registrar tiempo final.
end_time = time.time()

# Convertir resultados a una lista y mostrarlos.
resultados_lista = list(resultados)

for doc in resultados_lista:
    print(doc)

# Calcular y mostrar el tiempo de ejecución.
print(f"Tiempo de ejecución: {end_time - start_time:.4f} segundos")

{'_id': ObjectId('674769af1d7a4a76639b34a0'), 'identificador': 10, 'nombre': 'Priya', 'apellido_1': "O'Connor", 'apellido_2': 'Ivanov', 'posicion': 'Defensa', 'equipo': 'Borussia Dortmund', 'liga': 'Bundesliga', 'dorsal': 11, 'temporadas en equipo': 2, 'capitana': 'No', 'nacionalidad': 'Japonesa', 'anio de inicio': 2022, 'edad': 16, 'altura': 182, 'peso': 68, 'pierna dominante': 'Derecha', 'estadisticas': {'2022': {'goles': 0, 'asistencias': 11, 'porcentaje disparos gol': 63, 'porcentaje pases acertados': 98, 'porcentaje duelos aereos ganados': 48, 'velocidad maxima': 26, 'minutos jugados': 1201, 'tarjetas amarillas': 1, 'tarjetas rojas': 1, 'numero lesiones': 1, 'partidos como internacional': 12}, '2023': {'goles': 19, 'asistencias': 14, 'porcentaje disparos gol': 71, 'porcentaje pases acertados': 53, 'porcentaje duelos aereos ganados': 99, 'velocidad maxima': 27, 'minutos jugados': 3473, 'tarjetas amarillas': 4, 'tarjetas rojas': 3, 'numero lesiones': 1, 'partidos como internacional'

También es posile lanzar consultas más complejas, por ejemplo, veamos ahora qué jugadoras son defensas y juegan en el Manchester City.

In [183]:
# Registrar tiempo inicial.
start_time = time.time()

# Realizar la consulta.
resultados = players.find({"equipo": 'Manchester City', 'posicion': 'Defensa'})

# Registrar tiempo final.
end_time = time.time()

# Convertir resultados a una lista y mostrarlos.
resultados_lista = list(resultados)

for doc in resultados_lista:
    print(doc)

# Calcular y mostrar el tiempo de ejecución.
print(f"Tiempo de ejecución: {end_time - start_time:.4f} segundos")

{'_id': ObjectId('674769af1d7a4a76639b34aa'), 'identificador': 20, 'nombre': 'Carmen', 'apellido_1': 'Oliveira', 'apellido_2': 'Kim', 'posicion': 'Defensa', 'equipo': 'Manchester City', 'liga': 'Premier League', 'dorsal': 15, 'temporadas en equipo': 1, 'capitana': 'No', 'nacionalidad': 'Alemana', 'anio de inicio': 2020, 'edad': 18, 'altura': 165, 'peso': 59, 'pierna dominante': 'Derecha', 'estadisticas': {'2020': {'goles': 7, 'asistencias': 16, 'porcentaje disparos gol': 84, 'porcentaje pases acertados': 79, 'porcentaje duelos aereos ganados': 41, 'velocidad maxima': 30, 'minutos jugados': 1225, 'tarjetas amarillas': 8, 'tarjetas rojas': 1, 'numero lesiones': 0, 'partidos como internacional': 14}, '2021': {'goles': 0, 'asistencias': 16, 'porcentaje disparos gol': 70, 'porcentaje pases acertados': 87, 'porcentaje duelos aereos ganados': 69, 'velocidad maxima': 25, 'minutos jugados': 2981, 'tarjetas amarillas': 4, 'tarjetas rojas': 1, 'numero lesiones': 0, 'partidos como internacional': 

Otra posibilidad que ofrece MongoDB es la de mostrar únicamente los datos deseados de cada jugadora al devolver el resultado de una consulta. En este caso, queremos mostrar sólo las estadísticas de juego de la jugadora número 20 de la base de datos para todas sus temporadas como profesional.

In [184]:
# Registrar tiempo inicial.
start_time = time.time()

# Consultar estadísticas de juego de una jugadora concreta.
resultados = players.find({"identificador": 20},
                          {'estadisticas': 1, '_id': 0})

# Registrar tiempo final.
end_time = time.time()

# Convertir resultados a una lista y mostrarlos.
resultados_lista = list(resultados)

for doc in resultados_lista:
    print(doc)

# Calcular y mostrar el tiempo de ejecución.
print(f"Tiempo de ejecución: {end_time - start_time:.4f} segundos")

{'estadisticas': {'2020': {'goles': 7, 'asistencias': 16, 'porcentaje disparos gol': 84, 'porcentaje pases acertados': 79, 'porcentaje duelos aereos ganados': 41, 'velocidad maxima': 30, 'minutos jugados': 1225, 'tarjetas amarillas': 8, 'tarjetas rojas': 1, 'numero lesiones': 0, 'partidos como internacional': 14}, '2021': {'goles': 0, 'asistencias': 16, 'porcentaje disparos gol': 70, 'porcentaje pases acertados': 87, 'porcentaje duelos aereos ganados': 69, 'velocidad maxima': 25, 'minutos jugados': 2981, 'tarjetas amarillas': 4, 'tarjetas rojas': 1, 'numero lesiones': 0, 'partidos como internacional': 9}, '2022': {'goles': 17, 'asistencias': 18, 'porcentaje disparos gol': 73, 'porcentaje pases acertados': 70, 'porcentaje duelos aereos ganados': 34, 'velocidad maxima': 30, 'minutos jugados': 3207, 'tarjetas amarillas': 1, 'tarjetas rojas': 1, 'numero lesiones': 2, 'partidos como internacional': 0}, '2023': {'goles': 7, 'asistencias': 18, 'porcentaje disparos gol': 52, 'porcentaje pases 

Finalmente, aterrizando todo esto al caso de uso planteado en el documento, supóngase que nuestro equipo busca una delantera de cara a la presente temporada. Necesitamos fijarnos en jugadoras que hayan marcado más de 15 goles y dado más de 10 asistencias el pasado año, para decidir cuál de ellas sería la más adecuada para nosotros. Para ello, podría lanzarse una consulta como la mostrada debajo, que nos permite ver nombre, apellidos, posición y estadísticas de goles y asistencias de aquellas jugadoras que cumplen nuestras condiciones.

In [185]:
# Registrar tiempo inicial.
start_time = time.time()

# Consultar jugadoras que hayan marcado más de 15 goles y dado más de 10 asistencias el pasado año.
resultados = players.find({'estadisticas.2022.goles': {'$gt': 15}, 'estadisticas.2023.asistencias': {'$gt': 10}, 'posicion': 'Delantera'},
                          {'_id': 0, 'nombre': 1, 'apellido_1': 1, 'apellido_2': 1, 'equipo': 1, 'posicion': 1,
                           'estadisticas.2022.goles': 1, 'estadisticas.2022.asistencias': 1})

# Registrar tiempo final.
end_time = time.time()

# Convertir resultados a una lista y mostrarlos.
resultados_lista = list(resultados)

for doc in resultados_lista:
    print(doc)

# Calcular y mostrar el tiempo de ejecución.
print(f"Tiempo de ejecución: {end_time - start_time:.4f} segundos")

{'nombre': 'Aisha', 'apellido_1': 'Dupont', 'apellido_2': 'Muller', 'posicion': 'Delantera', 'equipo': 'Chelsea FC', 'estadisticas': {'2022': {'goles': 16, 'asistencias': 12}}}
{'nombre': 'Zofia', 'apellido_1': 'Ivanov', 'apellido_2': 'Patel', 'posicion': 'Delantera', 'equipo': 'Chelsea FC', 'estadisticas': {'2022': {'goles': 20, 'asistencias': 13}}}
{'nombre': 'Priya', 'apellido_1': 'Muller', 'apellido_2': 'Haddad', 'posicion': 'Delantera', 'equipo': 'Juventus', 'estadisticas': {'2022': {'goles': 19, 'asistencias': 12}}}
Tiempo de ejecución: 0.0001 segundos


## Actualización de datos

MongoDB también ofrece la posibilidad de modificar los datos una vez se han subido a la base de datos. Para ilustrar esto, tomamos dos jugadoras (la número 1 y la número 20 de la base de datos en este caso), y modificamos sus nombres para que estén en mayúsculas. Debajo de la celda, se muestra que en efecto se han modificado dos valores y el proceso se ha realizado correctamente.

In [201]:
filter = {'identificador': {'$in': [1, 20]}}

for doc in players.find(filter):

    update = {'$set':  {'nombre': doc['nombre'].upper()}}
    players.update_one({'_id': doc['_id']}, update)

players.update_many(filter, update)

UpdateResult({'n': 2, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Para visualizar el resultado, lanzamos una consulta para que se nos muestren los nombres que acabamos de modificar. Como podemos ver, todo ha ido según lo deseado.

In [204]:
# Registrar tiempo inicial.
start_time = time.time()

# Comprobamos si los nombres de las jugadoras están en mayúsculas.
resultados = players.find({"identificador": {'$in': [1, 2]}},
                          {'nombre': 1, '_id': 0})

# Registrar tiempo final.
end_time = time.time()

# Convertir resultados a una lista y mostrarlos.
resultados_lista = list(resultados)

for doc in resultados_lista:
    print(doc)

# Calcular y mostrar el tiempo de ejecución.
print(f"Tiempo de ejecución: {end_time - start_time:.4f} segundos")

{'nombre': 'CARMEN'}
{'nombre': 'AISHA'}
Tiempo de ejecución: 0.0001 segundos
